#Code for inference

In [16]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00


In [4]:
#Commenting the code where analyzing the tone is necessary

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForCausalLM


In [2]:
df = pd.read_csv("/content/tone_dataset.csv")

In [10]:
df.columns

Index(['original_email', 'Tone', 'passive-aggressiv'], dtype='object')

In [ ]:
# Step 1: Load and Preprocess the Email Dataset
def load_email_dataset(df):
    """Load dataset from DataFrame."""
    return df['original_email'].tolist() 

def preprocess_emails(emails):
    """Basic preprocessing: lowercase and strip."""
    return [email.lower().strip() for email in emails]

In [ ]:
# Step 2: Load Models

rewriter_tokenizer = AutoTokenizer.from_pretrained('gpt2')  

rewriter_tokenizer.pad_token = rewriter_tokenizer.eos_token
rewriter_model = AutoModelForCausalLM.from_pretrained('gpt2')  # Replace with LLaMA if you have it
rewriter_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Step 4: Define a Function to Rewrite Emails into Passive-Aggressive Tone
#def rewrite_email(email_text, original_tone):
def rewrite_email(email_text):
    """Use generative model to rewrite email in a passive-aggressive tone."""
    prompt = f"Rewrite this email in a passive-aggressive tone: {email_text}"
    inputs = rewriter_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = rewriter_model.generate(
            **inputs,
            max_length=200,
            temperature=0.7,
            num_return_sequences=1,
            do_sample=True,
            pad_token_id=rewriter_tokenizer.eos_token_id  
        )
    rewritten_text = rewriter_tokenizer.decode(outputs[0], skip_special_tokens=True)
    rewritten_text = rewritten_text.replace(prompt, "").strip()
    return rewritten_text

In [8]:
# Step 5: Process Each Email and Rewrite into Passive-Aggressive Tone
def process_emails(df):
    """Main function to process emails from DataFrame."""
    emails = load_email_dataset(df)
    preprocessed_emails = preprocess_emails(emails)

    rewritten_emails = []

    for email in preprocessed_emails:
        #tone = analyze_tone(email)
        #new_email = rewrite_email(email, tone)
        new_email = rewrite_email(email)
        rewritten_emails.append({
            "original": email,
            "rewritten": new_email,
            #"original_tone": tone
        })

    return rewritten_emails

In [ ]:
# Processing the DataFrame directly
results = process_emails(df.head(2))



In [13]:
results

[{'original': 'please submit your report by tomorrow.',
  'rewritten': 'Thank you.\n\nFrom:susan.chuck@washpost.com To: john.podesta@gmail.com Date: 2015-12-28 15:50 Subject: Re: Re: The DNC email dump On Wed, Dec 28, 2015 at 6:14 PM, SUSAN.CHUCK <susan.chuck@washpost.com> wrote: > > I think that\'s fine, but I\'m not sure this >> > I think that\'s fine, but I\'m not sure this >> > >> > > > On Wed, Dec 28, 2015 at 6:14 PM, "Susan.Chuck" <susan.chuck@washpost.com> wrote: >> >> >> >> > >> > >> Thanks for the reply. I was just wondering if we would be able to >> >> >> >> >> >> >'},
 {'original': 'can you attend the meeting at 3 pm?',
  'rewritten': 'Reply With Quote'}]

#Fine Tunning GPT

In [23]:
df.columns

Index(['original_email', 'Tone', 'passive-aggressiv'], dtype='object')

In [17]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [ ]:
# Preparing dataset for fine-tuning
def prepare_finetuning_dataset(df):
    """Convert DataFrame to a format suitable for fine-tuning."""
    data = [
        {"input": row["original_email"], "output": row["passive-aggressiv"]}
        for _, row in df.iterrows()
    ]
    return Dataset.from_list(data)

In [ ]:
# Tokenize dataset
def tokenize_dataset(dataset, tokenizer):
    """Tokenize inputs and outputs for fine-tuning."""
    def tokenize_function(examples):
        inputs = [f"Rewrite this email in a passive-aggressive tone: {x}" for x in examples["input"]]
        outputs = examples["output"]
        model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)
        labels = tokenizer(outputs, truncation=True, padding="max_length", max_length=128)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    return dataset.map(tokenize_function, batched=True)

In [ ]:
# Fine-tune the model

def finetune_model(model, tokenizer, dataset, output_dir="finetuned_model"):
    training_args = TrainingArguments(
    output_dir = output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    logging_strategy="steps",  
    logging_steps=10,          
    log_level="info",         
    report_to=[]
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=data_collator,
    )
    trainer.train()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model fine-tuned and saved to {output_dir}")

In [31]:
# Update main execution
dataset = prepare_finetuning_dataset(df)
tokenized_dataset = tokenize_dataset(dataset, rewriter_tokenizer)
finetune_model(rewriter_model, rewriter_tokenizer, tokenized_dataset)

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

***** Running training *****
  Num examples = 147
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 222
  Number of trainable parameters = 124,439,808
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: output, input. If output, input are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
10,0.710600
20,0.719700
30,0.742300
40,0.772300
50,0.866400
60,0.805700
70,0.748900
80,0.676800
90,0.514000
100,0.598200


Saving model checkpoint to finetuned_model/checkpoint-222
Configuration saved in finetuned_model/checkpoint-222/config.json
Configuration saved in finetuned_model/checkpoint-222/generation_config.json
Model weights saved in finetuned_model/checkpoint-222/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in finetuned_model/config.json
Configuration saved in finetuned_model/generation_config.json
Model weights saved in finetuned_model/model.safetensors
tokenizer config file saved in finetuned_model/tokenizer_config.json
Special tokens file saved in finetuned_model/special_tokens_map.json


Model fine-tuned and saved to finetuned_model


In [28]:


# After fine-tuning, use the model for inference as before
rewriter_model = AutoModelForCausalLM.from_pretrained("finetuned_model")
rewriter_tokenizer = AutoTokenizer.from_pretrained("finetuned_model")